<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## How can I analyze supply, and ongoing price change behavior of the total supply in a given market?

In this notebook, we will be looking for supply stressors that may indicate downward momentum for home prices. Specifically analyzing YoY supply trends relative to price cut changes.

#### What will you create in this notebook?

<p align="center">
  <img src="../../../images/tampa_market_price_drops.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/supply_and_demand/supply_and_price_changes.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [1]:
import os
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.charting.default_charts import create_dual_axis_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Markets

In [2]:
# in this case, lets look at Tampa market at the metro level
tampa_market = client.search.markets.retrieve(
    query='Tampa',
    sort_by='TOTAL_POPULATION',
    location_type='CBSA',
    sort_order='DESC',
    limit=1
)

tampa_market.head()

,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,2900417,USA,45300,None,"Tampa-St. Petersburg-Clearwater, Fl",None,None,CBSA,3194310,67197,0,1,0,1


In [3]:
tampa_parcl_id = tampa_market['parcl_id'].values[0]

### 3. Retrieve the Data

In [4]:
# Lets get YoY changes for tampa market

start_date='2023-06-01'

supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=[tampa_parcl_id],
    limit=60,
    start_date=start_date
)

supply_price_changes = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    parcl_ids=[tampa_parcl_id],
    limit=60,
    start_date=start_date
)

|████████████████████████████████████████| 1/1 [100%] in 0.1s (8.02/s) 
|████████████████████████████████████████| 1/1 [100%] in 0.1s (8.27/s) 


In [5]:
supply.head()

,date,for_sale_inventory,parcl_id,property_type
0,2024-06-17,22001,2900417,ALL_PROPERTIES
1,2024-06-10,22070,2900417,ALL_PROPERTIES
2,2024-06-03,21925,2900417,ALL_PROPERTIES
3,2024-05-27,22282,2900417,ALL_PROPERTIES
4,2024-05-20,22463,2900417,ALL_PROPERTIES


In [6]:
supply_price_changes.head()

,date,count_price_change,count_price_drop,median_days_bt_change,median_price_change,median_pct_price_change,pct_inventory_price_change,parcl_id,property_type
0,2024-06-17,11151,10575,22,-10000,-2.54,50.68,2900417,ALL_PROPERTIES
1,2024-06-10,10927,10374,22,-10000,-2.57,49.51,2900417,ALL_PROPERTIES
2,2024-06-03,10811,10268,23,-10000,-2.60,49.31,2900417,ALL_PROPERTIES
3,2024-05-27,10813,10261,23,-10000,-2.61,48.53,2900417,ALL_PROPERTIES
4,2024-05-20,10717,10172,24,-10000,-2.61,47.71,2900417,ALL_PROPERTIES


### 4. Prepare the data for analysis/charting

In [7]:
# lets focus on price drops, combine total supply with count of price drops
data = pd.merge(supply, supply_price_changes[['parcl_id', 'date', 'count_price_drop']], on=['date', 'parcl_id'], how='left')

In [8]:
data['% Price Drops'] = round(data['count_price_drop'] / data['for_sale_inventory'] * 100, 2)
data = data.rename(columns={'for_sale_inventory': 'Total Supply'})

### 5. Chart the Data

In [9]:
data = data.sort_values('date')

In [10]:
from parcllabs.beta.charting.styling import default_style_config

default_style_config['tick_prefix'] = ''

create_dual_axis_chart(
    title='Tampa Metro: % of Inventory with Price Drops vs Total Supply',
    line_data=data,
    bar1_data=data,
    line_series='% Price Drops',
    bar1_series='Total Supply',
    yaxis1_title='% Price Drops',
    yaxis2_title='Total Supply',
    height=SIZE_CONFIG['x']['height'],
    width=SIZE_CONFIG['x']['width'],
    style_config=default_style_config,
    # save_path='tampa_market_price_drops.png' # to save figure
)